# LGG Brain MRI Segmentation — Transfer Learning + Self-Attention (ResNet-U-Net)

This notebook trains a segmentation model on the LGG FLAIR abnormality dataset:
- Patient-level split to prevent leakage
- Transfer learning (ImageNet pretrained ResNet34 encoder)
- Bottleneck self-attention (multi-head attention on deep features)
- Dice + BCE loss, Dice/IoU metrics
- Paper-style plots (global matplotlib styling)

Outputs saved to `/kaggle/working/`.


In [ ]:
# If Kaggle already has these, this cell is harmless.
!pip -q install albumentations==1.3.1 opencv-python-headless==4.9.0.80


In [ ]:
import os, re, glob, random, math, time
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models import resnet34, ResNet34_Weights

import albumentations as A
from albumentations.pytorch import ToTensorV2


In [ ]:
# -------------------------------
# ✅ GLOBAL PAPER-STYLE SETTINGS (as you provided)
# -------------------------------
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
    "font.size": 12,

    "axes.labelsize": 13,
    "axes.titlesize": 13,
    "axes.linewidth": 1.2,

    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "xtick.major.size": 6,
    "ytick.major.size": 6,
    "xtick.minor.size": 3,
    "ytick.minor.size": 3,

    "legend.fontsize": 11,
    "legend.frameon": True,
    "legend.edgecolor": "0.4",

    "grid.linestyle": ":",
    "grid.linewidth": 0.7,
    "grid.alpha": 0.85,
})

def paper_axes(ax):
    ax.minorticks_on()
    ax.grid(True, which="major", linestyle=":", linewidth=0.8)
    ax.grid(True, which="minor", linestyle=":", linewidth=0.5, alpha=0.7)

    for spine in ax.spines.values():
        spine.set_linewidth(1.2)

    ax.tick_params(which="both", direction="in", top=True, right=True)

# -------------------------------
# Reproducibility
# -------------------------------
SEED = 7
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = False  # faster; OK with fixed seed
torch.backends.cudnn.benchmark = True


In [ ]:
CFG = {
    "img_size": 256,
    "batch_size": 16,
    "num_workers": 2,
    "epochs": 12,
    "lr": 3e-4,
    "weight_decay": 1e-4,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "use_attention": True,          # ablation knob
    "attn_heads": 8,                # 512 channels / 8 heads works cleanly
    "threshold": 0.5,
}

CFG


In [ ]:
## Load dataset index (with safe fallback)

Preferred:
- Load `lgg_master_slices.csv` created by your EDA notebook.

Fallback:
- Rebuild the dataframe directly from `/kaggle/input/**/kaggle_3m`.


In [ ]:
KAGGLE_INPUT = Path("/kaggle/input")
candidates = list(KAGGLE_INPUT.glob("**/kaggle_3m"))
print("Found candidates:", [str(p) for p in candidates[:10]])

if len(candidates) == 0:
    raise FileNotFoundError("Could not find 'kaggle_3m' under /kaggle/input. Attach the dataset to this notebook.")

DATA_ROOT = candidates[0]
DATA_ROOT


In [ ]:
def to_key(path):
    base = Path(path).name
    if base.endswith("_mask.tif"):
        base = base.replace("_mask.tif", "")
    else:
        base = base.replace(".tif", "")
    return base

def patient_id_from_path(path):
    return Path(path).parent.name

def slice_index_from_key(k):
    m = re.search(r"_(\d+)$", k)
    return int(m.group(1)) if m else np.nan

EDA_CSV = Path("/kaggle/working/eda_outputs/lgg_master_slices.csv")

if EDA_CSV.exists():
    df = pd.read_csv(EDA_CSV)
    print("Loaded:", EDA_CSV, "| rows:", len(df), "| patients:", df["patient_id"].nunique())
else:
    all_tifs = sorted(glob.glob(str(DATA_ROOT / "*" / "*.tif")))
    mask_tifs = sorted([p for p in all_tifs if p.endswith("_mask.tif")])
    img_tifs  = sorted([p for p in all_tifs if not p.endswith("_mask.tif")])

    img_map = {to_key(p): p for p in img_tifs}
    msk_map = {to_key(p): p for p in mask_tifs}
    keys = sorted(set(img_map.keys()) & set(msk_map.keys()))

    df = pd.DataFrame({
        "key": keys,
        "image_path": [img_map[k] for k in keys],
        "mask_path":  [msk_map[k] for k in keys],
    })
    df["patient_id"] = df["image_path"].apply(patient_id_from_path)
    df["slice_idx"] = df["key"].apply(slice_index_from_key)

    print("Built df | rows:", len(df), "| patients:", df["patient_id"].nunique())

df.head()


In [ ]:
if "split" not in df.columns or df["split"].isna().any():
    patients = df["patient_id"].drop_duplicates().tolist()
    rng = np.random.default_rng(SEED)
    rng.shuffle(patients)

    n = len(patients)
    train_pat = set(patients[: int(0.8*n)])
    val_pat   = set(patients[int(0.8*n): int(0.9*n)])
    test_pat  = set(patients[int(0.9*n):])

    def assign_split(pid):
        if pid in train_pat: return "train"
        if pid in val_pat: return "val"
        return "test"

    df["split"] = df["patient_id"].apply(assign_split)

df["split"].value_counts(), df.groupby("split")["patient_id"].nunique()


## Dataset + augmentations

Notes:
- Images are 256×256 and stored as 3-channel uint8 (we normalize to [0,1]).
- Masks are binarized to {0,1} and returned as a 1×H×W tensor.


In [ ]:
train_tfms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.04, scale_limit=0.08, rotate_limit=12, p=0.6, border_mode=0),
    A.RandomBrightnessContrast(p=0.25),
    A.GaussianBlur(blur_limit=3, p=0.15),
    A.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),  # keep simple; MR intensity isn't ImageNet-like
    ToTensorV2(),
])

val_tfms = A.Compose([
    A.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),
    ToTensorV2(),
])


In [ ]:
def load_tif(path):
    return np.array(Image.open(path))

def to_mask01(msk):
    if msk.ndim == 3:
        m = msk[..., 0]
    else:
        m = msk
    return (m > 0).astype(np.uint8)

class LGGSegDataset(Dataset):
    def __init__(self, df, tfms=None):
        self.df = df.reset_index(drop=True)
        self.tfms = tfms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        img = load_tif(r.image_path)          # (H,W,3) uint8
        msk = to_mask01(load_tif(r.mask_path))# (H,W) uint8 {0,1}

        # Ensure 3 channels
        if img.ndim == 2:
            img = np.stack([img, img, img], axis=-1)

        # Albumentations expects mask HxW or HxWx1
        if self.tfms is not None:
            out = self.tfms(image=img, mask=msk)
            img_t = out["image"]  # (3,H,W) float
            msk_t = out["mask"]   # (H,W)
        else:
            img = img.astype(np.float32) / 255.0
            img_t = torch.from_numpy(img).permute(2,0,1).float()
            msk_t = torch.from_numpy(msk).long()

        msk_t = msk_t.unsqueeze(0).float()   # (1,H,W)
        return img_t, msk_t


In [ ]:
train_df = df[df["split"]=="train"].copy()
val_df   = df[df["split"]=="val"].copy()
test_df  = df[df["split"]=="test"].copy()

train_ds = LGGSegDataset(train_df, tfms=train_tfms)
val_ds   = LGGSegDataset(val_df, tfms=val_tfms)
test_ds  = LGGSegDataset(test_df, tfms=val_tfms)

train_loader = DataLoader(train_ds, batch_size=CFG["batch_size"], shuffle=True,
                          num_workers=CFG["num_workers"], pin_memory=True, drop_last=True)
val_loader   = DataLoader(val_ds, batch_size=CFG["batch_size"], shuffle=False,
                          num_workers=CFG["num_workers"], pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=CFG["batch_size"], shuffle=False,
                          num_workers=CFG["num_workers"], pin_memory=True)

len(train_ds), len(val_ds), len(test_ds)


In [ ]:
xb, yb = next(iter(train_loader))
print("x:", xb.shape, xb.dtype, "y:", yb.shape, yb.dtype, "mask unique:", torch.unique(yb))

# Plot a few samples (paper style)
fig, ax = plt.subplots(figsize=(13, 6))
paper_axes(ax)
ax.axis("off")
ax.set_title("Training batch sanity-check (first sample)")

img = xb[0].permute(1,2,0).cpu().numpy()
msk = yb[0,0].cpu().numpy()

ax.imshow(np.clip(img, 0, 1))
ax.imshow(msk, alpha=0.35, cmap="Reds")
plt.tight_layout()
plt.show()


## Model: ResNet34 U-Net + bottleneck self-attention

- Encoder: ResNet34 pretrained on ImageNet (transfer learning)
- Bottleneck attention: multi-head self-attention over the 8×8 deep feature tokens
- Decoder: U-Net style upsampling with skip connections


In [ ]:
class ConvBNReLU(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, p=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=k, padding=p, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.net(x)

class UpBlock(nn.Module):
    def __init__(self, in_ch, skip_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2)
        self.conv1 = ConvBNReLU(out_ch + skip_ch, out_ch)
        self.conv2 = ConvBNReLU(out_ch, out_ch)

    def forward(self, x, skip):
        x = self.up(x)
        # handle any off-by-1 due to rounding (shouldn't happen at 256, but keep safe)
        if x.shape[-2:] != skip.shape[-2:]:
            x = F.interpolate(x, size=skip.shape[-2:], mode="bilinear", align_corners=False)
        x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class BottleneckMHSA(nn.Module):
    """
    Multi-head self-attention on deep feature map.
    Operates on (B,C,H,W) -> tokens (B,HW,C) -> MHA -> reshape back, residual + FFN.
    """
    def __init__(self, channels, heads=8, dropout=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(channels)
        self.attn = nn.MultiheadAttention(embed_dim=channels, num_heads=heads,
                                          dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(channels)
        self.ffn = nn.Sequential(
            nn.Linear(channels, channels*4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(channels*4, channels),
        )

    def forward(self, x):
        b, c, h, w = x.shape
        tokens = x.permute(0,2,3,1).reshape(b, h*w, c)  # (B,HW,C)

        t = self.norm1(tokens)
        attn_out, _ = self.attn(t, t, t, need_weights=False)
        tokens = tokens + attn_out

        t2 = self.norm2(tokens)
        tokens = tokens + self.ffn(t2)

        x_out = tokens.reshape(b, h, w, c).permute(0,3,1,2)
        return x_out


In [ ]:
class ResNet34UNetAttn(nn.Module):
    def __init__(self, pretrained=True, use_attention=True, attn_heads=8):
        super().__init__()
        weights = ResNet34_Weights.IMAGENET1K_V1 if pretrained else None
        self.encoder = resnet34(weights=weights)

        # ResNet stem
        self.conv1 = self.encoder.conv1
        self.bn1   = self.encoder.bn1
        self.relu  = self.encoder.relu
        self.maxp  = self.encoder.maxpool

        # Encoder stages (skip connections)
        self.layer1 = self.encoder.layer1  # 64
        self.layer2 = self.encoder.layer2  # 128
        self.layer3 = self.encoder.layer3  # 256
        self.layer4 = self.encoder.layer4  # 512

        self.use_attention = use_attention
        self.attn = BottleneckMHSA(512, heads=attn_heads) if use_attention else nn.Identity()

        # Decoder
        self.up3 = UpBlock(512, 256, 256)
        self.up2 = UpBlock(256, 128, 128)
        self.up1 = UpBlock(128, 64, 64)

        # Stem skip: after conv1+bn+relu is 64 at 128x128
        self.up0 = UpBlock(64, 64, 64)

        self.out = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        # Encoder
        x0 = self.relu(self.bn1(self.conv1(x)))  # (B,64,128,128)
        x1 = self.layer1(self.maxp(x0))          # (B,64,64,64)
        x2 = self.layer2(x1)                     # (B,128,32,32)
        x3 = self.layer3(x2)                     # (B,256,16,16)
        x4 = self.layer4(x3)                     # (B,512,8,8)

        # Self-attention bottleneck
        x4 = self.attn(x4)

        # Decoder (upsample + skips)
        d3 = self.up3(x4, x3)                    # (B,256,16,16)
        d2 = self.up2(d3, x2)                    # (B,128,32,32)
        d1 = self.up1(d2, x1)                    # (B,64,64,64)
        d0 = self.up0(d1, x0)                    # (B,64,128,128)

        # Final upsample to 256x256
        d0 = F.interpolate(d0, scale_factor=2, mode="bilinear", align_corners=False)  # (B,64,256,256)

        logits = self.out(d0)                    # (B,1,256,256)
        return logits


In [ ]:
def dice_coeff(probs, targets, eps=1e-7):
    # probs, targets: (B,1,H,W) float in [0,1]
    num = 2.0 * (probs * targets).sum(dim=(2,3))
    den = (probs + targets).sum(dim=(2,3)) + eps
    return (num / den).mean()

def iou_coeff(probs, targets, eps=1e-7):
    inter = (probs * targets).sum(dim=(2,3))
    union = (probs + targets - probs*targets).sum(dim=(2,3)) + eps
    return (inter / union).mean()

class DiceLoss(nn.Module):
    def __init__(self, eps=1e-7):
        super().__init__()
        self.eps = eps
    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        num = 2.0 * (probs * targets).sum(dim=(2,3))
        den = (probs + targets).sum(dim=(2,3)) + self.eps
        dice = (num / den)
        return 1.0 - dice.mean()

bce = nn.BCEWithLogitsLoss()

def total_loss(logits, targets):
    return 0.6 * bce(logits, targets) + 0.4 * DiceLoss()(logits, targets)


In [ ]:
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    losses, dices, ious = [], [], []
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        logits = model(xb)
        loss = total_loss(logits, yb)

        probs = torch.sigmoid(logits)
        probs_bin = (probs > CFG["threshold"]).float()

        losses.append(loss.item())
        dices.append(dice_coeff(probs_bin, yb).item())
        ious.append(iou_coeff(probs_bin, yb).item())

    return {
        "loss": float(np.mean(losses)),
        "dice": float(np.mean(dices)),
        "iou":  float(np.mean(ious)),
    }

def train_one_epoch(model, loader, optimizer, scaler, device):
    model.train()
    losses = []
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            logits = model(xb)
            loss = total_loss(logits, yb)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        losses.append(loss.item())

    return float(np.mean(losses))


In [ ]:
device = CFG["device"]
model = ResNet34UNetAttn(
    pretrained=True,
    use_attention=CFG["use_attention"],
    attn_heads=CFG["attn_heads"],
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))

best_val_dice = -1.0
history = []

save_path = Path("/kaggle/working/best_resnet34_unet_attn.pt")

for epoch in range(1, CFG["epochs"]+1):
    t0 = time.time()
    train_loss = train_one_epoch(model, train_loader, optimizer, scaler, device)
    val_stats = evaluate(model, val_loader, device)

    row = {
        "epoch": epoch,
        "train_loss": train_loss,
        "val_loss": val_stats["loss"],
        "val_dice": val_stats["dice"],
        "val_iou": val_stats["iou"],
        "sec": time.time() - t0
    }
    history.append(row)

    if val_stats["dice"] > best_val_dice:
        best_val_dice = val_stats["dice"]
        torch.save({"model": model.state_dict(), "cfg": CFG}, save_path)

    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} | val_loss={val_stats['loss']:.4f} | "
          f"val_dice={val_stats['dice']:.4f} | val_iou={val_stats['iou']:.4f} | best_dice={best_val_dice:.4f}")

history_df = pd.DataFrame(history)
history_df.tail()
